In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd
import json

In [2]:
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression

In [7]:
label_encoding = {
"istj":1,
"istp":2,
"isfj":3,
"isfp":4,
"infj":5,
"infp":6,
"intj":7,
"intp":8,
"estp":9,
"estj":10,
"esfp":11,
"esfj":12,
"enfp":13,
"enfj":14,
"entp":15,
"entj":16
}

In [24]:
train_tagged_data = []
count = 0
with open('/home/amoghmishra23/appledore/elmo/data/train_elmo_10000.json','r') as f:
    for i in f:
        doc = json.loads(i)
        train_tagged_data.append(TaggedDocument(doc['text'], tags=[doc['label']]))
        count+=1
#         if count == 1000:
#             break

In [25]:
test_tagged_data = []
count = 0
with open('/home/amoghmishra23/appledore/elmo/data/test_elmo_10000.json','r') as f:
    for i in f:
        doc = json.loads(i)
        test_tagged_data.append(TaggedDocument(doc['text'], tags=[doc['label']]))
        count+=1
#         if count == 100:
#             break

In [27]:
test_tagged_data[0]

TaggedDocument(words="['really', 'consider', 'it', 'to', 'be', 'morally', 'equivalent', 'to', 'other', 'things', 'then', 'my', 'argument', 'would', 'have', 'no', 'effect', 'on', 'him', 'if', 'ted', 'bundy', 'made', 'such', 'statement', 'we', 'can', 'imagine', 'it', 'as', 'being', 'detached', 'of', 'preference', 'my', 'argument', 'is', 'about', 'calling', 'contrarians', 'as', 'pseudo', 'moral', 'relativists', 'its', 'about', 'getting', 'them', 'to', 'admit', 'they', 'actually', 'have', 'a', 'sense', 'of', 'right', 'and', 'wrong', 'by', 'admitting', 'they', 'have', 'a', 'sense', 'of', 'right', 'and', 'wrong', 'they', 'admit', 'to', 'the', 'dichotomy', 'between', 'right', 'and', 'wrong', 'status<number>', 'not', 'to', 'anyone', 'who', 'took', 'seriously', 'all', 'the', 'evidence', 'there', 'already', 'was', 'who', 'is', 'this', 'cute', 'and', 'smart', 'woman', 'with', 'a', 'blue', 'shirt', 'on', 'cnn', 'posts<number>they', 'want', 'charges', 'dropped', 'on', 'two', 'black', 'students', 'w

In [ ]:
max_epochs = 20
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(train_tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(train_tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("../model/d2v_10kk.model")
print("Model Saved")

iteration 0


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6


In [6]:
similar_doc = model.docvecs.most_similar(8)
print(similar_doc)

[(15, 0.51785808801651), (16, 0.4996769428253174), (9, 0.4302116632461548), (12, 0.3153681755065918), (6, 0.2842196226119995), (5, 0.25804468989372253), (14, 0.2516555190086365), (1, 0.19906215369701385), (7, 0.15495653450489044), (2, 0.11726222932338715)]


In [35]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [11]:
model= Doc2Vec.load("../model/d2v_10k.model")

In [12]:
def vec_for_learning(model, sents):
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [13]:
y_train, X_train = vec_for_learning(model, train_tagged_data)

In [14]:
y_test, X_test = vec_for_learning(model, test_tagged_data)

In [15]:
logreg = LogisticRegression(n_jobs=4, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score, f1_score

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.32070566388115135
Testing F1 score: 0.24290970484736688


In [19]:
import pickle

In [23]:
pickle.dump((y_train,X_train), open("../data/train_ml_vec.p", "wb" ) )
pickle.dump((y_test,X_test), open("../data/test_ml_vec.p", "wb" ) )